In [6]:
!pip install ultralytics fastai opencv-python-headless ffmpeg-python

In [7]:
import cv2
from fastai.vision.all import load_learner
from ultralytics import YOLO
import ffmpeg
import tempfile
import os


def process_video(input_video_path, output_video_path, model_path, confidence_threshold=0.5, skip_interval=0.5):
    # Load YOLO face detection model
    yolo_model = YOLO('/content/yolov8n-face.pt')  # YOLO model for face detection

    # Load the FastAI classification model
    learner = load_learner(model_path)

    # Temporary file for annotated video without audio
    temp_video_file = tempfile.NamedTemporaryFile(delete=False, suffix='.mp4')

    # Open the input video file
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        raise FileNotFoundError(f"Unable to open video file: {input_video_path}")

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))  # Frames per second
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # Calculate frames to skip
    frames_to_skip = int(fps * skip_interval)

    # Initialize video writer for annotated frames
    out = cv2.VideoWriter(temp_video_file.name, fourcc, fps, (frame_width, frame_height))

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Process only specific frames
        if frame_count % frames_to_skip == 0:
            # Convert frame to RGB for YOLO
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Detect faces using YOLO
            results = yolo_model(frame_rgb)

            # Annotate detected faces
            for box in results[0].boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = box.conf[0]

                # Filter detections by confidence score
                if confidence < confidence_threshold:
                    continue

                # Crop the detected face
                face = frame_rgb[y1:y2, x1:x2]
                if face.size == 0 or (y2 - y1) < 20 or (x2 - x1) < 20:
                    continue

                # Resize face for classification
                face_resized = cv2.resize(face, (224, 224))

                # Predict the class using FastAI model
                pred, _, probs = learner.predict(face_resized)

                # Annotate frame with bounding box and label
                label = f"{pred} ({probs.max():.2f})"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Bounding box
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)  # Label

        # Write the frame (annotated or skipped) to the output video
        out.write(frame)
        frame_count += 1

    # Release resources
    cap.release()
    out.release()

    # Use ffmpeg-python to add audio back to the video
    input_video = ffmpeg.input(input_video_path)
    annotated_video = ffmpeg.input(temp_video_file.name)

    # Combine video (from temp file) with audio (from original input)
    ffmpeg.output(annotated_video.video, input_video.audio, output_video_path, vcodec='libx264', acodec='aac').run()

    # Cleanup temporary file
    os.remove(temp_video_file.name)

    print(f"Processed video saved at: {output_video_path}")


In [8]:
if __name__ == "__main__":
    # Input parameters
    input_video_path = "/content/bbnoinput.mp4"  # Path to the input video
    output_video_path = "/content/bbnooutput.mp4"  # Path to save the processed video
    model_path = "/content/children_vs_adults_model.pkl"  # Path to the trained FastAI model

    # Process the video
    process_video(input_video_path, output_video_path, model_path)


0: 384x640 18 faces, 19.1ms
Speed: 2.7ms preprocess, 19.1ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)



0: 384x640 17 faces, 12.1ms
Speed: 2.5ms preprocess, 12.1ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)



0: 384x640 19 faces, 11.5ms
Speed: 2.5ms preprocess, 11.5ms inference, 5.7ms postprocess per image at shape (1, 3, 384, 640)



0: 384x640 18 faces, 11.9ms
Speed: 4.6ms preprocess, 11.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)



0: 384x640 16 faces, 13.1ms
Speed: 2.1ms preprocess, 13.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)


...


0: 384x640 4 faces, 11.3ms
Speed: 2.5ms preprocess, 11.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)



0: 384x640 4 faces, 11.6ms
Speed: 5.7ms preprocess, 11.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)



0: 384x640 3 faces, 13.2ms
Speed: 4.3ms preprocess, 13.2ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)



0: 384x640 1 face, 11.3ms
Speed: 2.3ms preprocess, 11.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)



0: 384x640 (no detections), 14.3ms
Speed: 2.6ms preprocess, 14.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 18.2ms
Speed: 3.0ms preprocess, 18.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 15.9ms
Speed: 3.6ms preprocess, 15.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.9ms
Speed: 2.1ms preprocess, 11.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 14.2ms
Speed: 2.1ms preprocess, 14.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.9ms
Speed: 3.0ms preprocess, 10.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.9ms
Speed: 2.1ms preprocess, 10.9ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 13.7ms
Speed: 2.1ms preprocess, 13.7ms i